In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/forest-cover-type-prediction/test.csv
/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv
/kaggle/input/forest-cover-type-prediction/train.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import seaborn as sns
from IPython.display import display
%matplotlib inline

import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_222"; OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1~deb9u1-b10); OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp42ftizjh
  JVM stdout: /tmp/tmp42ftizjh/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp42ftizjh/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 20 days
H2O cluster name:,H2O_from_python_unknownUser_yrhscm
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


# Read Dataset

In [3]:
from sklearn.model_selection import train_test_split

data_train = pd.read_csv('/kaggle/input/forest-cover-type-prediction/train.csv', index_col='Id')
data_test = pd.read_csv('/kaggle/input/forest-cover-type-prediction/test.csv', index_col='Id')

# Reversed One-Hot Encoding
Turn one-hot encoding of `Soil_TypeXX` to categories.

In [4]:
keywords  = ['soil']
all_columns = data_train.columns
col_keywords = []

for k in keywords:
    for col in all_columns:
        if k.lower() in col.lower(): 
            col_keywords.append(col)
            
            
sequences = list(np.arange(1, 1+len(col_keywords)))
soil_type_dict = dict(zip(['Soil_Type{}'.format(n) for n in sequences],
                        sequences))

index_soil = data_train.columns.get_loc('Soil_Type40')
data_train.insert(index_soil+1, 'Soil_Types', np.nan)

data_train['Soil_Types'] = data_train[col_keywords].idxmax(axis=1)
# data_valid['Soil_Types'] = data_valid[col_keywords].idxmax(axis=1)
data_test['Soil_Types'] = data_test[col_keywords].idxmax(axis=1)

data_train['Soil_Types'] = data_train['Soil_Types'].map(soil_type_dict)
# data_valid['Soil_Types'] = data_valid['Soil_Types'].map(soil_type_dict)
data_test['Soil_Types'] = data_test['Soil_Types'].map(soil_type_dict)

In [5]:
data_train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Soil_Types,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,29,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,29,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,12,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,30,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,29,5


#### Check which `Soil_TypeXX` is missing

In [6]:
set(range(1, 40+1)) - set(data_train['Soil_Types'].unique())

{7, 15}

# Target Encoding

In [7]:
crosstab = pd.crosstab( data_train['Soil_Types'], data_train['Cover_Type'])

# display(crosstab)

# crosstab.sum(axis=1)

In [8]:
crosstab_average = crosstab.T / crosstab.sum(axis=1)
display(crosstab_average)

Soil_Types,1,2,3,4,5,6,8,9,10,11,...,31,32,33,34,35,36,37,38,39,40
Cover_Type,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.005931,0.000000,0.000000,0.0,0.1,0.004202,0.012315,...,0.343373,0.333333,0.298701,0.000000,0.029412,0.0,0.0,0.109890,0.120244,0.106754
2,0.000000,0.004815,0.012474,0.023725,0.000000,0.010769,1.0,0.9,0.037815,0.165025,...,0.292169,0.369565,0.298701,0.545455,0.009804,0.2,0.0,0.009615,0.004566,0.006536
3,0.340845,0.454254,0.138254,0.548043,0.333333,0.381538,0.0,0.0,0.334734,0.219212,...,0.000000,0.007246,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
4,0.391549,0.150883,0.830561,0.157770,0.236364,0.375385,0.0,0.0,0.079365,0.059113,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
5,0.000000,0.097913,0.000000,0.153025,0.000000,0.000000,0.0,0.0,0.029879,0.379310,...,0.262048,0.144928,0.186688,0.181818,0.000000,0.0,0.0,0.000000,0.000000,0.000000
6,0.267606,0.292135,0.018711,0.103203,0.430303,0.232308,0.0,0.0,0.514006,0.165025,...,0.021084,0.043478,0.107143,0.090909,0.000000,0.0,0.0,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.008304,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.081325,0.101449,0.108766,0.181818,0.960784,0.8,1.0,0.880495,0.875190,0.886710


In [9]:
index_cover_type = data_train.columns.get_loc('Soil_Types')

col_soil_mean_enc = []

for i_cover in range(1, 7+1):
    col_name = 'Soil_mean_enc{}'.format(i_cover)
    col_soil_mean_enc.append(col_name)
    data_train.insert(index_cover_type+i_cover, col_name, np.nan)
    
print(col_soil_mean_enc)

['Soil_mean_enc1', 'Soil_mean_enc2', 'Soil_mean_enc3', 'Soil_mean_enc4', 'Soil_mean_enc5', 'Soil_mean_enc6', 'Soil_mean_enc7']


In [10]:
data_train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type40,Soil_Types,Soil_mean_enc1,Soil_mean_enc2,Soil_mean_enc3,Soil_mean_enc4,Soil_mean_enc5,Soil_mean_enc6,Soil_mean_enc7,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [11]:
crosstab_average.iloc[:, 0]

Cover_Type
1    0.000000
2    0.000000
3    0.340845
4    0.391549
5    0.000000
6    0.267606
7    0.000000
Name: 1, dtype: float64

In [12]:
data_train[col_soil_mean_enc] = data_train['Soil_Types'].apply(lambda soil_type: crosstab_average.loc[:, soil_type])

# a = data_train['Soil_Types'].map(lambda soil_type: crosstab_average.loc[:, soil_type])
# print(a.shape)
# print(a.iloc[0])

In [13]:
data_train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type40,Soil_Types,Soil_mean_enc1,Soil_mean_enc2,Soil_mean_enc3,Soil_mean_enc4,Soil_mean_enc5,Soil_mean_enc6,Soil_mean_enc7,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,29,0.315259,0.429125,0.0,0.0,0.196747,0.0,0.058869,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,29,0.315259,0.429125,0.0,0.0,0.196747,0.0,0.058869,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,12,0.105727,0.894273,0.0,0.0,0.000000,0.0,0.000000,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,30,0.111724,0.198621,0.0,0.0,0.662069,0.0,0.027586,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,29,0.315259,0.429125,0.0,0.0,0.196747,0.0,0.058869,5


# Drop One-Hot Encodded `Soil_TypeXX` 

In [14]:
import re

# col_soil_types = [col for col in data_train.columns if 'Soil_Type' in col]
# print(col_soil_types)

col_soil_types = []

for col in data_train.columns:
    m = re.match('Soil_Type[\d]+', col)
    if m is not None:
#         print(m.group(0))
        col_soil_types.append(m.group(0))
    
print(len(col_soil_types))


data_train = data_train.drop(columns=col_soil_types)
data_test = data_test.drop(columns=col_soil_types)

40


In [15]:
print(data_train.columns)
print()
print(data_test.columns)

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Types', 'Soil_mean_enc1', 'Soil_mean_enc2', 'Soil_mean_enc3',
       'Soil_mean_enc4', 'Soil_mean_enc5', 'Soil_mean_enc6', 'Soil_mean_enc7',
       'Cover_Type'],
      dtype='object')

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Types'],
      dtype='object')


# Train / Valid Splitting

In [16]:
df_train, df_valid = train_test_split(data_train, test_size=0.1, random_state=42)

htrain_frame = h2o.H2OFrame(df_train)
hvalid_frame = h2o.H2OFrame(df_valid)
htest_frame = h2o.H2OFrame(data_test)
# htrain_frame = h2o.H2OFrame(data_train.drop(columns=col_keywords))
# hvalid_frame = h2o.H2OFrame(data_valid.drop(columns=col_keywords))
# htest_frame = h2o.H2OFrame(data_test.drop(columns=col_keywords))

y = 'Cover_Type'

htrain_frame[y] = htrain_frame[y].asfactor()
hvalid_frame[y] = hvalid_frame[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Start Training

In [17]:
%%time
aml = h2o.estimators.random_forest.H2ORandomForestEstimator(
                            max_runtime_secs=60, 
#                            max_models=1,
                            balance_classes=True,
                           seed=42)

aml.train(y=y, training_frame=htrain_frame)

drf Model Build progress: |███████████████████████████████████████████████| 100%
CPU times: user 368 ms, sys: 32 ms, total: 400 ms
Wall time: 10.2 s


In [18]:
aml.model_performance()


ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.12252742579601729
RMSE: 0.3500391775159136
LogLoss: 0.5701933705758313
Mean Per-Class Error: 0.1364214155996276
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



1,2,3,4,5,6,7,Error,Rate
1489.0,316.0,2.0,0.0,42.0,5.0,107.0,0.2406935,"472 / 1,961"
312.0,1380.0,45.0,0.0,154.0,51.0,14.0,0.2944785,"576 / 1,956"
0.0,8.0,1599.0,93.0,21.0,228.0,0.0,0.1795793,"350 / 1,949"
0.0,0.0,33.0,1908.0,0.0,20.0,0.0,0.0270270,"53 / 1,961"
5.0,44.0,26.0,0.0,1859.0,23.0,0.0,0.0500766,"98 / 1,957"
0.0,18.0,171.0,50.0,11.0,1707.0,0.0,0.1277466,"250 / 1,957"
65.0,3.0,1.0,0.0,0.0,0.0,1883.0,0.0353484,"69 / 1,952"
1871.0,1769.0,1877.0,2051.0,2087.0,2034.0,2004.0,0.1364201,"1,868 / 13,693"


Top-7 Hit Ratios: 


k,hit_ratio
1,0.8635799
2,0.9744395
3,0.9915285
4,0.9942306
5,0.9946688
6,0.9946688
7,1.0


In [19]:
# predictions = aml.predict(hvalid_frame.drop(y))

# accuracy = accuracy_score(data_valid[y],
#                          predictions['predict'].as_data_frame())

# print('Accuracy: {}'.format(accuracy))

# Prediction

In [20]:
%%time
prediction_test_hframe = aml.predict(htest_frame)

submission = pd.DataFrame.from_dict({'ID': data_test.index.tolist(),
                                    'Cover_Type': prediction_test_hframe['predict'].as_data_frame().iloc[:,0].tolist(),
                                    })

submission.to_csv('./submission.csv', index=False)

drf prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Soil_mean_enc1': substituting in a column of NaN
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Soil_mean_enc2': substituting in a column of NaN
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Soil_mean_enc3': substituting in a column of NaN
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Soil_mean_enc4': substituting in a column of NaN
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'Soil_mean_enc5': substituting in a column of NaN
  warnings.warn(w)
/opt/conda/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column '

CPU times: user 3.76 s, sys: 476 ms, total: 4.24 s
Wall time: 39.5 s


In [21]:
print(submission.head())

print()
!head submission.csv

      ID  Cover_Type
0  15121           2
1  15122           1
2  15123           1
3  15124           2
4  15125           2

ID,Cover_Type
15121,2
15122,1
15123,1
15124,2
15125,2
15126,2
15127,2
15128,2
15129,2
